In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import nltk
import os
import re
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import Activation

from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Embedding, Conv1D, MaxPooling1D, Bidirectional,
                                     LSTM, Dense, Dropout, BatchNormalization)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import Add
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.preprocessing.text import Tokenizer


In [ ]:
# 1. Randomly sample 100,000 observations from your data
data_sample = data.sample(n=10000, random_state=42)

# 2. Apply the preprocessing steps to this subset
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(data_sample['Lyrics_Processed'])

word_index = tokenizer.word_index
max_words = len(word_index) + 1

sequences = tokenizer.texts_to_sequences(data_sample['Lyrics_Processed'])
padded_sequences = pad_sequences(sequences, maxlen=30, truncating='post', padding='post')

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(data_sample['genre'])
labels_one_hot = to_categorical(labels_encoded, num_classes=6)
labels = labels_one_hot

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

maxlen = 64
embedding_dim = 32

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.regularizers import l2

model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=30))
model.add(Dropout(0.5))

# First LSTM layer with dropout and kernel regularization
model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.4, kernel_regularizer=l2(0.05)))
model.add(Dropout(0.4))

# Intermediate LSTM layer with dropout and kernel regularization
model.add(LSTM(1024, return_sequences=True, dropout=0.2, recurrent_dropout=0.4, kernel_regularizer=l2(0.05)))
model.add(Dropout(0.4))

# Second LSTM layer with dropout and kernel regularization
model.add(LSTM(64, dropout=0.4, recurrent_dropout=0.2, kernel_regularizer=l2(0.05)))
model.add(Dropout(0.4))

# Dense layer with kernel regularization
model.add(Dense(32, activation='tanh', kernel_regularizer=l2(0.05)))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()
